<a href="https://colab.research.google.com/github/orevs-com/Stock-Market-Price-Prediction/blob/main/data_preprocessing_v3.ipynb%20%E2%80%93%20Raw%20data%20cleaning%20and%20feature%20engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Enhanced Stock Price Prediction Using LSTM with Technical Indicators: A Comparative Study with XGBoost and ARIMA

# by Orevaoghene Otiede

Importing the required libraries for the project


In [1]:
!pip uninstall numpy pmdarima -y
!pip install numpy==1.26.4

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
Found existing installation: pmdarima 2.0.4
Uninstalling pmdarima-2.0.4:
  Successfully uninstalled pmdarima-2.0.4
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [1]:
# Install pandas_ta
!pip install pandas_ta statsmodels xgboost
!pip install pmdarima

  Using cached pmdarima-2.0.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_28_x86_64.whl.metadata (7.8 kB)
Using cached pmdarima-2.0.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_28_x86_64.whl (2.2 MB)


In [2]:
# Corrcting the numpy import for pandas_ta
path = "/usr/local/lib/python3.11/dist-packages/pandas_ta/momentum/squeeze_pro.py"

# Read the file and replace the line
with open(path, 'r') as file:
    content = file.read()

# Replace the incorrect import
content = content.replace("from numpy import NaN as npNaN", "from numpy import nan as npNaN")

# Write the updated content back
with open(path, 'w') as file:
    file.write(content)

print("File updated successfully.")

File updated successfully.


In [16]:
#Importing the Libraries
import numpy as np
import pandas as pd
import pandas_ta as ta
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import linear_model
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
import keras.backend as K
from pmdarima import auto_arima
import yfinance as yf
import os
from statsmodels.tsa.arima.model import ARIMA as PyARIMA
import warnings
import pickle
import xgboost as xgb

In [4]:
# Import the drive module from the google.colab package.
from google.colab import drive

# Mount the Google Drive to the specified mount point in the Colab environment.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


List of ticker symbols for Training Companies

In [5]:
# Create a list of Ticker Symbols
train_tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'NVDA', 'META']
test_tickers = ['TSLA', 'AVGO', 'COST', 'NFLX', 'ADBE', 'INTC']

Downloading and Saving the Stock Market Price Data

In [6]:
# Define the date range
start_date = "2015-01-01"
end_date = "2024-12-31"
# Define the file path
drive_path = '/content/drive/MyDrive/Colab Notebooks/Stock Market Data/'

# Function to download and save data for a list of tickers
def download_and_save_tickers(tickers, start, end, base_path):
    for ticker in tickers:
        try:
            # Dwonload data from Yahoo Finance with auto adjusted closed price
            df = yf.download(ticker, start=start, end=end, interval="1d",
                             auto_adjust=True)

            if not df.empty:
                file_path = os.path.join(base_path, f"{ticker}.csv")
                df.to_csv(file_path)
                print(f"Successfully downloaded and saved data for {ticker}")
            else:
                print(f"No data found for {ticker} in the specified date range.")

        except Exception as e:
            print(f"Error downloading data for {ticker}: {e}")

# Download and save training data
download_and_save_tickers(train_tickers, start_date, end_date, drive_path)

# Download and save testing data
download_and_save_tickers(test_tickers, start_date, end_date, drive_path)

print("\nAll data download attempts complete.")

[*********************100%***********************]  1 of 1 completed


Successfully downloaded and saved data for AAPL


[*********************100%***********************]  1 of 1 completed


Successfully downloaded and saved data for MSFT


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

...

Successfully downloaded and saved data for NFLX


[*********************100%***********************]  1 of 1 completed


Successfully downloaded and saved data for ADBE


[*********************100%***********************]  1 of 1 completed

Successfully downloaded and saved data for INTC

All data download attempts complete.


Calculating for Technical Indicators

In [7]:
# Defining the directory where the stock data CSV files are saved
data_dir = '/content/drive/MyDrive/Colab Notebooks/Stock Market Data/'
output_dir = '/content/drive/MyDrive/Colab Notebooks/Data with Indicators/'

# Create an Output Directory
os.makedirs(output_dir, exist_ok=True)

# Get a list of all CSV files
csv_files = [f for f in os.listdir(data_dir) if f.endswith('.csv')]

for file_name in csv_files:
    ticker = file_name.replace('.csv', '') # Extract ticker from file name
    file_path = os.path.join(data_dir, file_name)

    try:
        df = pd.read_csv(file_path, skiprows=3) # Considering Multi Index Rows
        df.columns = ['Date', 'Close', 'High', 'Low', 'Open', 'Volume']
        df['Date'] = pd.to_datetime(df['Date'])
        df.set_index('Date', inplace=True)
        print(f"Calculating indicators for {ticker}...")

        if not all(col in df.columns for col in ['Open', 'High', 'Low', 'Volume']):
            print(f"Skipping {ticker}: Missing OHLCV columns after parsing.")
            continue

        # Calculating Technical Indicators using pandas_ta

        # Trend Indicators
        df.ta.sma(length=200, append=True)
        df.ta.sma(length=50, append=True)
        df.ta.ema(length=26, append=True)
        df.ta.macd(append=True)

        # Momentum Indicators
        df.ta.rsi(length=14, append=True)
        df.ta.willr(append=True)

        # Volatility Indicator
        df.ta.bbands(append=True)

        # Volume Indicators
        df.ta.obv(append=True)
        df.ta.cmf(append=True)

        # Adding Daily Returns and Log Returns
        df['Daily_Return'] = df['Close'].pct_change()
        df['Log_Return'] = np.log(df['Close'] / df['Close'].shift(1))
        df.replace([np.inf, -np.inf], np.nan, inplace=True)

        # Add Lagged Features for all calculated indicators

        # Columns from SMA (Trend Indicators)
        lag_cols = ['SMA_200', 'SMA_50', 'EMA_26']

        # Columns from MACD (Trend Indicators)
        lag_cols.extend(['MACD_12_26_9', 'MACDh_12_26_9', 'MACDs_12_26_9'])

        # Columns from RSI, Williams (Momentum Indicators)
        lag_cols.extend(['RSI_14', 'WILLR_14'])

        # Columns from Bollinger Bands (Volatiity Indicators)
        lag_cols.extend(['BBL_5_2.0', 'BBM_5_2.0', 'BBU_5_2.0', 'BBB_5_2.0', 'BBP_5_2.0'])

        # Columns from OBV, CMF (Volume Indicators)
        lag_cols.extend(['OBV', 'CMF_20'])

        # Columns from Daily Returns and Log Returns
        lag_cols.extend(['Daily_Return', 'Log_Return'])

        # Apply lagging
        for col in lag_cols:
            if col in df.columns:
                df[f'Lag_{col}'] = df[col].shift(1)
            else:
                print(f"  Column '{col}' not found for {ticker} .")

         # Data Cleaning after adding Technical Indicators and Lagged Features
        rows = len(df)
        df.dropna(inplace=True)
        print(f"  Dropped {rows - len(df)} rows with NaN values for {ticker}.")

        # Save the processed DataFrame to a new CSV
        output_file_path = os.path.join(output_dir, f"{ticker}_indicators.csv")
        df.to_csv(output_file_path)
        print(f"Saved Technical Indicators for {ticker} to {output_file_path}")

    except Exception as e:
        print(f"Error processing {ticker}: {e}")

Calculating indicators for GOOGL...
  Dropped 200 rows with NaN values for GOOGL.
Saved Technical Indicators for GOOGL to /content/drive/MyDrive/Colab Notebooks/Data with Indicators/GOOGL_indicators.csv
Calculating indicators for AMZN...
  Dropped 200 rows with NaN values for AMZN.
Saved Technical Indicators for AMZN to /content/drive/MyDrive/Colab Notebooks/Data with Indicators/AMZN_indicators.csv
Calculating indicators for NVDA...
  Dropped 200 rows with NaN values for NVDA.
Saved Technical Indicators for NVDA to /content/drive/MyDrive/Colab Notebooks/Data with Indicators/NVDA_indicators.csv
Calculating indicators for AVGO...
  Dropped 200 rows with NaN values for AVGO.
Saved Technical Indicators for AVGO to /content/drive/MyDrive/Colab Notebooks/Data with Indicators/AVGO_indicators.csv
Calculating indicators for META...
  Dropped 200 rows with NaN values for META.
Saved Technical Indicators for META to /content/drive/MyDrive/Colab Notebooks/Data with Indicators/META_indicators.csv
C

Data Preprocessing

In [17]:
# Suppress warnings from pmdarima
warnings.filterwarnings("ignore")

# Defining the directory where the stock data CSV files are saved
output_dir = '/content/drive/MyDrive/Colab Notebooks/Data with Indicators/'
processed_dir = '/content/drive/MyDrive/Colab Notebooks/Processed Data/'

# Create an Output Directory
os.makedirs(processed_dir, exist_ok=True)

# Structure for Data Preparation
TARGET_COLUMN = 'Close' # Next day's Adjusted Close
SPLIT_DATE = '2023-01-01' # Date to split training and testing data
LSTM_LOOKBACK = 60 # Number of past days to look at for LSTM sequences

# Define ticker for feature selection
FEATURE_SELECTION_TICKER = 'AAPL'

# Dictionary to store prepared data for each model and each stock
all_prepared_data = {
    'lstm': {},
    'xgboost': {},
    'arima': {}
}

# Get all processed CSV files
files = [f for f in os.listdir(output_dir) if f.endswith('_indicators.csv')]

# Feature selection to determine best Indicators

# Load data for selected feature selection ticker
fs_path = os.path.join(output_dir, f"{FEATURE_SELECTION_TICKER}_indicators.csv")
if not os.path.exists(fs_path):
    print(f"Feature selection ticker {FEATURE_SELECTION_TICKER}_indicators.csv not found")

fs_df = pd.read_csv(fs_path, index_col='Date', parse_dates=True)

# Define Target and Features for feature selection model
fs_df['Target'] = fs_df[TARGET_COLUMN].shift(-1)
fs_exclude_cols = ['Close', 'Target']
fs_feature_cols = [col for col in fs_df.columns if pd.api.types.is_numeric_dtype(fs_df[col]) and col not in fs_exclude_cols]

fs_X_df = fs_df[fs_feature_cols]
fs_y_df = fs_df['Target']

# Clean NaNs for feature selection data
fs_combined_df = pd.concat([fs_X_df, fs_y_df], axis=1).dropna()
fs_X_clean = fs_combined_df[fs_feature_cols]
fs_y_clean = fs_combined_df['Target']

if fs_X_clean.empty or fs_y_clean.empty:
    print(f"Not enough data for {FEATURE_SELECTION_TICKER} for feature selection")

# Split data for feature selection model
fs_X_train = fs_X_clean[fs_X_clean.index < SPLIT_DATE]
fs_y_train = fs_y_clean[fs_y_clean.index < SPLIT_DATE]

if fs_X_train.empty or fs_y_train.empty:
    print(f"Training data for {FEATURE_SELECTION_TICKER} empty.")

# Scale features for XGBoost using MinMaxScaler
fs_scaler_X = MinMaxScaler(feature_range=(0, 1))
fs_X_train_scaled = fs_scaler_X.fit_transform(fs_X_train)
fs_X_train_scaled_df = pd.DataFrame(fs_X_train_scaled, columns=fs_X_train.columns, index=fs_X_train.index)


# Train a simple XGBoost Regressor for feature importance
print(f" Training temporary XGBoost model on {FEATURE_SELECTION_TICKER} to get feature importances...")
model_xgb_fs = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, random_state=42)
model_xgb_fs.fit(fs_X_train_scaled_df, fs_y_train)

# Get feature importances
feature_importances = model_xgb_fs.feature_importances_
features_df = pd.DataFrame({'Feature': fs_X_train_scaled_df.columns, 'Importance': feature_importances})
features_df = features_df.sort_values(by='Importance', ascending=False)

print("\n  Top 10 Features by Importance:")
print(features_df.head(10))

# Select the top 3 features
selected_features = features_df['Feature'].head(3).tolist()
print(f"\n  Selected Top 3 Features for all models: {selected_features}")

# Data Preparation using selected features

for file_name in files:
    ticker = file_name.replace('_indicators.csv', '')
    file_path = os.path.join(output_dir, file_name)

    try:
        df = pd.read_csv(file_path, index_col='Date', parse_dates=True)

        # Define Target Variable (Y Axis)
        df['Target'] = df[TARGET_COLUMN].shift(-1)

        # Feature Selection ( X Axis)
        three_selected_features = [f for f in selected_features if f in df.columns and pd.api.types.is_numeric_dtype(df[f])]

        if len(three_selected_features) < 3:
            print(f"Not all top 3 features found in {ticker}. Using {len(three_selected_features)} features.")
            if not three_selected_features:
                 print(f"Skipping {ticker}: No selected features found.")
                 continue


        X_df = df[three_selected_features]
        y_df = df['Target']

        # Dropping any other NaNs
        combined_df = pd.concat([X_df, y_df], axis=1).dropna()
        X_clean = combined_df[three_selected_features]
        y_clean = combined_df['Target']

        if X_clean.empty or y_clean.empty:
            print(f"Skipping {ticker}: Not enough clean data after NaN removal. (Shape X: {X_clean.shape}, Y: {y_clean.shape})")
            continue
        print(f"Cleaned data shape: X={X_clean.shape}, y={y_clean.shape}")

        # Train-Test Split
        X_train = X_clean[X_clean.index < SPLIT_DATE]
        X_test = X_clean[X_clean.index >= SPLIT_DATE]
        y_train = y_clean[y_clean.index < SPLIT_DATE]
        y_test = y_clean[y_clean.index >= SPLIT_DATE]

        if X_train.empty or X_test.empty or y_train.empty or y_test.empty:
            print(f"Skipping {ticker}: Train or test set is empty.")
            continue

        print(f"Train/Test split: Train X={X_train.shape}, y={y_train.shape} | Test X={X_test.shape}, y={y_test.shape}")

        # Feature Scaling
        scaler_X = MinMaxScaler(feature_range=(0, 1))
        X_train_scaled = scaler_X.fit_transform(X_train)
        X_test_scaled = scaler_X.transform(X_test)

        scaler_y = MinMaxScaler(feature_range=(0, 1))
        y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1))
        y_test_scaled = scaler_y.transform(y_test.values.reshape(-1, 1))

        # Model-Specific Data Preparation

        # LSTM Data Preparation
        train_generator = TimeseriesGenerator(X_train_scaled, y_train_scaled.flatten(),
                                              length=LSTM_LOOKBACK, batch_size=1)
        test_generator = TimeseriesGenerator(X_test_scaled, y_test_scaled.flatten(),
                                             length=LSTM_LOOKBACK, batch_size=1)

        X_train_lstm = np.array([train_generator[i][0][0] for i in range(len(train_generator))])
        y_train_lstm = np.array([train_generator[i][1] for i in range(len(train_generator))])
        X_test_lstm = np.array([test_generator[i][0][0] for i in range(len(test_generator))])
        y_test_lstm = np.array([test_generator[i][1] for i in range(len(test_generator))])

        X_train_lstm = X_train_lstm.reshape(X_train_lstm.shape[0], LSTM_LOOKBACK, X_train_scaled.shape[1])
        X_test_lstm = X_test_lstm.reshape(X_test_lstm.shape[0], LSTM_LOOKBACK, X_test_scaled.shape[1])

        print(f"    LSTM Train X shape: {X_train_lstm.shape}, y shape: {y_train_lstm.shape}")
        print(f"    LSTM Test X shape: {X_test_lstm.shape}, y shape: {y_test_lstm.shape}")

        all_prepared_data['lstm'][ticker] = {
            'X_train': X_train_lstm, 'y_train': y_train_lstm,
            'X_test': X_test_lstm, 'y_test': y_test_lstm,
            'scaler_X': scaler_X, 'scaler_y': scaler_y
        }

        # XGBoost Data Preparation
        X_train_xgb = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
        X_test_xgb = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)

        y_train_xgb = y_train
        y_test_xgb = y_test

        print(f"    XGBoost Train X shape: {X_train_xgb.shape}, y shape: {y_train_xgb.shape}")
        print(f"    XGBoost Test X shape: {X_test_xgb.shape}, y shape: {y_test_xgb.shape}")

        all_prepared_data['xgboost'][ticker] = {
            'X_train': X_train_xgb, 'y_train': y_train_xgb,
            'X_test': X_test_xgb, 'y_test': y_test_xgb,
            'scaler_X': scaler_X
        }

        # ARIMA Data Preparation
        arima_series_train = y_train.copy()
        arima_series_test = y_test.copy()

        arima_order = None
        try:
            stepwise_fit = auto_arima(arima_series_train, start_p=1, start_q=1,
                                      max_p=5, max_q=5, m=1,
                                      d=None, seasonal=False,
                                      trace=False, error_action='ignore', suppress_warnings=True,
                                      stepwise=True)
            arima_order = stepwise_fit.order
            print(f"    Auto-ARIMA found order (p,d,q): {arima_order}")
        except Exception as arima_e:
            print(f"    Could not determine ARIMA order with auto_arima for {ticker}: {arima_e}")
            print("    Defaulting to (5,1,0) for ARIMA. Consider manually inspecting ACF/PACF plots.")
            arima_order = (5, 1, 0)

        all_prepared_data['arima'][ticker] = {
            'train_series': arima_series_train,
            'test_series': arima_series_test,
            'order': arima_order,
            'scaler_y': scaler_y
        }

    except Exception as e:
        print(f"Error processing {ticker}: {e}")

try:
    processed_file_path = os.path.join(processed_dir, 'all_prepared_stock_data.pkl')
    with open(processed_file_path, 'wb') as f:
        pickle.dump(all_prepared_data, f)
    print(f"\nAll prepared data saved as a pickle file to: {processed_file_path}")
except Exception as e:
    print(f"Error saving all_prepared_data: {e}")


 Training temporary XGBoost model on AAPL to get feature importances...

  Top 10 Features by Importance:
          Feature  Importance
30  Lag_BBM_5_2.0    0.601825
4         SMA_200    0.167547
31  Lag_BBU_5_2.0    0.087977
1             Low    0.057373
0            High    0.036961
14      BBU_5_2.0    0.029919
2            Open    0.008398
29  Lag_BBL_5_2.0    0.003489
5          SMA_50    0.002464
13      BBM_5_2.0    0.001876

  Selected Top 3 Features for all models: ['Lag_BBM_5_2.0', 'SMA_200', 'Lag_BBU_5_2.0']
Cleaned data shape: X=(2313, 3), y=(2313,)
Train/Test split: Train X=(1813, 3), y=(1813,) | Test X=(500, 3), y=(500,)
    LSTM Train X shape: (1753, 60, 3), y shape: (1753, 1)
    LSTM Test X shape: (440, 60, 3), y shape: (440, 1)
    XGBoost Train X shape: (1813, 3), y shape: (1813,)
    XGBoost Test X shape: (500, 3), y shape: (500,)
    Auto-ARIMA found order (p,d,q): (0, 1, 1)
Cleaned data shape: X=(2313, 3), y=(2313,)
Train/Test split: Train X=(1813, 3), y=(1813,) |